In [ ]:
# 測試：查詢英雄分析 (以 Ahri 中路為例, champion_id=103)
print("📊 查詢 Ahri 中路的英雄分析...\n")

result = await client.get_champion_analysis(
    region="kr",
    champion_id=103,  # Ahri
    lane="MID"
)

print("結果:")
print("=" * 60)
print(result)

## 9. 測試：查詢英雄分析 (Counter/Ban/Pick)

In [ ]:
# 測試：查詢英雄列表
print("🎮 查詢英雄列表...\n")

result = await client.list_champions(region="kr")

print("結果:")
print("=" * 60)
print(result)

## 8. 測試：查詢英雄列表

In [ ]:
# 測試：查詢 Faker (Hide on bush) 的資料
print("🔍 查詢 Faker (Hide on bush) 的資料...\n")

result = await client.get_summoner_profile(
    region="kr",
    summoner_id="Hide on bush",
    game_type="SOLORANKED"
)

print("結果:")
print("=" * 60)
print(result)

## 7. 測試：查詢召喚師資料

In [ ]:
# 測試：列出所有工具
tools = await client.list_tools()

print(f"✅ 成功連接！找到 {len(tools)} 個工具\n")
print("工具列表:")
print("=" * 60)

for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}")
    if hasattr(tool, 'description'):
        print(f"   {tool.description[:100]}...")
    print()

## 6. 測試：列出所有工具

In [ ]:
class SimpleMCPClient:
    """
    簡單的 MCP Client 封裝類別
    """
    
    def __init__(self, server_url: str = "https://mcp-api.op.gg/mcp"):
        self.server_url = server_url
        self.server_params = StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "supergateway",
                "--streamableHttp",
                server_url
            ],
            env=None
        )
    
    async def list_tools(self):
        """
        列出所有可用的工具
        """
        async with stdio_client(self.server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                tools_result = await session.list_tools()
                return tools_result.tools
    
    async def call_tool(self, tool_name: str, arguments: dict):
        """
        呼叫指定的工具
        
        Args:
            tool_name: 工具名稱
            arguments: 工具參數
        
        Returns:
            工具執行結果
        """
        async with stdio_client(self.server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                result = await session.call_tool(tool_name, arguments=arguments)
                return result
    
    async def get_summoner_profile(self, region: str, summoner_id: str, game_type: str = "SOLORANKED"):
        """
        便利方法：查詢召喚師資料
        """
        return await self.call_tool(
            "lol_get_summoner_profile",
            {
                "region": region,
                "summoner_id": summoner_id,
                "game_type": game_type
            }
        )
    
    async def list_champions(self, region: str = "kr"):
        """
        便利方法：查詢英雄列表
        """
        return await self.call_tool(
            "lol_list_champions",
            {"region": region}
        )
    
    async def get_champion_analysis(self, region: str, champion_id: int, lane: str):
        """
        便利方法：查詢英雄分析
        """
        return await self.call_tool(
            "lol_get_champion_analysis",
            {
                "region": region,
                "champion_id": champion_id,
                "lane": lane
            }
        )

# 創建 client 實例
client = SimpleMCPClient()
print("✅ SimpleMCPClient 已創建")

## 5. 完整的 MCP Client 類別封裝

In [ ]:
async def call_mcp_tool_example():
    """
    範例：使用 MCP client 呼叫工具
    """
    server_params = StdioServerParameters(
        command="npx",
        args=[
            "-y",
            "supergateway",
            "--streamableHttp",
            "https://mcp-api.op.gg/mcp"
        ],
        env=None
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # 範例 1: 查詢召喚師資料
            print("🔍 查詢召喚師資料...\n")
            
            result = await session.call_tool(
                "lol_get_summoner_profile",
                arguments={
                    "region": "kr",
                    "summoner_id": "Hide on bush",
                    "game_type": "SOLORANKED"
                }
            )
            
            print("📊 結果:")
            print(result)
            print()
            
            # 範例 2: 查詢英雄列表
            print("\n🎮 查詢英雄列表...\n")
            
            result2 = await session.call_tool(
                "lol_list_champions",
                arguments={
                    "region": "kr"
                }
            )
            
            print("📊 結果:")
            print(result2)

# 執行範例 - 取消下面這行的註解來執行
# await call_mcp_tool_example()

## 4. 呼叫 MCP 工具範例

In [10]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def create_mcp_client():
    """
    建立並初始化 MCP client，連接到 OP.GG MCP server
    """
    # 設定 server 參數 (使用 npx supergateway 連接到 OP.GG)
    server_params = StdioServerParameters(
        command="npx",
        args=[
            "-y",
            "supergateway",
            "--streamableHttp",
            "https://mcp-api.op.gg/mcp"
        ],
        env=None
    )
    
    # 建立 stdio client 和 session
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # 初始化連接
            await session.initialize()
            
            print("✅ 成功連接到 MCP server!")
            
            # 列出所有可用的工具
            tools_result = await session.list_tools()
            
            result = await session.call_tool(
				"lol_get_champion_analysis",
				arguments={
					"champion": "ELISE",
					"desired_value_description": "champion, win_rate, pick_rate, ban_rate, recommended_build, skill_order, items, counter_pick, weaknesses, recommended_synergies",
					"game_mode": "RANKED",
					"lang": "zh_TW",
					"position": "JUNGLE"
				}
			)
            
            print("📊 結果:")
            print(result)
            # tools = tools_result.tools
            
            # print(f"\n📦 可用工具數量: {len(tools)}\n")
            
            # for i, tool in enumerate(tools, 1):
            #     print(f"{i}. {tool.name}")
            #     print(f"   描述: {tool.description}")
            #     print()
            


            return session, tools

# 執行 - 取消下面這行的註解來執行
session, tools = await create_mcp_client()

CancelledNotification.method
  Input should be 'notifications/cancelled' [type=literal_error, input_value='notifications/initialized', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
CancelledNotification.params
  Field required [type=missing, input_value={'method': 'notifications...ized', 'jsonrpc': '2.0'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
ProgressNotification.method
  Input should be 'notifications/progress' [type=literal_error, input_value='notifications/initialized', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
ProgressNotification.params
  Field required [type=missing, input_value={'method': 'notifications...ized', 'jsonrpc': '2.0'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
LoggingMessageNotification.method
  Input should be 'notifications/message' [type=literal_err

✅ 成功連接到 MCP server!
📊 結果:
meta=None content=[TextContent(type='text', text='{"champion":"ELISE","position":"JUNGLE","win_rate":0.52,"pick_rate":0.04,"ban_rate":0.04,"recommended_build":{"core_items":[{"ids":"[4646,4645,3157]","play":1031,"win":527,"pick_rate":0.2}],"boots":[{"ids":"[3020]","play":9181,"win":4717,"pick_rate":0.98}],"starter_items":[{"ids":"[1101,2003]","play":3618,"win":1875,"pick_rate":0.37}],"last_items":[{"ids":"[4646]","play":8253,"win":4227,"pick_rate":0.8},{"ids":"[4645]","play":7316,"win":3872,"pick_rate":0.71},{"ids":"[1082]","play":3699,"win":1926,"pick_rate":0.36}],"fourth_items":[{"ids":"[3089]","play":503,"win":319,"pick_rate":0.25},{"ids":"[3135]","play":470,"win":266,"pick_rate":0.23},{"ids":"[3157]","play":434,"win":289,"pick_rate":0.22}],"fifth_items":[{"ids":"[3135]","play":127,"win":75,"pick_rate":0.25},{"ids":"[3089]","play":91,"win":59,"pick_rate":0.18},{"ids":"[3102]","play":81,"win":40,"pick_rate":0.16}],"sixth_items":[{"ids":"[3102]","play":8,"win

## 3. 建立 MCP Client 連接到 OP.GG MCP Server

## 2. 導入必要的模組

In [ ]:
# 安裝 fastmcp 和 mcp (如果還沒安裝)
!pip install fastmcp mcp

## 1. 安裝必要套件

首先確保已安裝 fastmcp 和 mcp

# FastMCP Client 範例

這個 notebook 展示如何使用 fastmcp 建立一個 MCP client 來連接 MCP server